In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\1\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\1\test'
checkpoint_path = r'C:\Users\24306\Desktop\EfficientNetV2L\1\EfficientNetV2L.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[10.0, 80.0, 37.0, 47.0, 45.0, 23.0, 16.0, 52.0, 37.0, 16.0, 96.0, 37.0, 95.0, 20.0, 86.0, 86.0, 35.0, 20.0, 95.0, 13.0, 96.0, 95.0, 37.0, 45.0, 96.0, 66.0, 40.0, 100.0, 52.0, 66.0, 35.0, 47.0, 10.0, 66.0, 47.0, 86.0, 32.0, 47.0, 84.0, 13.0, 16.0, 84.0, 40.0, 10.0, 13.0, 95.0, 10.0, 37.0, 80.0, 40.0, 7.0, 37.0, 100.0, 96.0, 47.0, 40.0, 95.0, 40.0, 95.0, 40.0, 84.0, 47.0, 66.0, 55.0, 80.0, 13.0, 84.0, 40.0, 7.0, 80.0, 52.0, 13.0, 7.0, 66.0, 37.0, 20.0, 20.0, 23.0, 37.0, 10.0, 66.0, 40.0, 80.0, 35.0, 4.0, 37.0, 10.0, 23.0, 86.0, 35.0, 100.0, 66.0, 100.0, 80.0, 23.0, 80.0, 86.0, 23.0, 96.0, 84.0, 96.0, 86.0, 66.0, 84.0, 55.0, 20.0, 45.0, 37.0, 80.0, 96.0, 52.0, 84.0, 95.0, 16.0, 23.0, 47.0, 40.0, 52.0, 40.0, 96.0, 52.0, 86.0, 45.0, 45.0, 32.0, 39.0, 37.0, 16.0, 13.0, 80.0, 95.0, 4.0, 45.0, 100.0, 66.0, 100.0, 93.0, 32.0, 84.0, 10.0, 52.0, 86.0, 80.0, 37.0, 30.0, 66.0, 80.0, 35.0, 80.0, 55.0, 86.0, 66.0, 96.0, 47.0, 37.0, 16.0, 23.0, 80.0, 96.0, 95.0, 13.0, 52.0, 13.0, 80.0, 96.0, 7.0, 100

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
EfficientNetV2L = tf.keras.applications.efficientnet_v2.EfficientNetV2L(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
EfficientNetV2L.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = EfficientNetV2L(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-l (Functiona  (None, 7, 7, 1280)       117746848 
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 1281      
                                                                 
Total params: 117,748,129
Trainable params: 117,235,553
Non-trainable params: 512,576
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
#要改
np.savetxt('EfficientNetV2L_loss_1.txt',history.history['loss'])
#要改
np.savetxt('EfficientNetV2L_valloss_1.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Wed May  3 10:13:22 2023
Epoch 1/300
591/591 [==============================] - ETA: 0s - loss: 1056.6342

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\1\EfficientNetV2L.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\1\EfficientNetV2L.ckpt\assets


591/591 [==============================] - 543s 876ms/step - loss: 1056.6342 - val_loss: 428.2625
Epoch 2/300
591/591 [==============================] - ETA: 0s - loss: 322.2961

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\1\EfficientNetV2L.ckpt\assets


...

591/591 [==============================] - 1113s 2s/step - loss: 0.3563 - val_loss: 0.4687
Epoch 140/300
591/591 [==============================] - 887s 2s/step - loss: 0.4843 - val_loss: 4.7149
Epoch 141/300
591/591 [==============================] - 885s 1s/step - loss: 0.5443 - val_loss: 1.8980
Epoch 142/300
591/591 [==============================] - 907s 2s/step - loss: 0.3108 - val_loss: 2.6685
Epoch 143/300
591/591 [==============================] - 897s 2s/step - loss: 0.4795 - val_loss: 1.0220
Epoch 144/300
591/591 [==============================] - 874s 1s/step - loss: 0.7255 - val_loss: 13.6366
Epoch 145/300
591/591 [==============================] - 737s 1s/step - loss: 0.3479 - val_loss: 0.5662
Epoch 146/300
591/591 [==============================] - 730s 1s/step - loss: 0.4823 - val_loss: 3.4669
Epoch 147/300
591/591 [==============================] - 730s 1s/step - loss: 0.2565 - val_loss: 0.5239
Epoch 148/300
591/591 [==============================] - 730s 1s/step - loss

Epoch 218/300
591/591 [==============================] - 777s 1s/step - loss: 0.3402 - val_loss: 2.2068
Epoch 219/300
591/591 [==============================] - 777s 1s/step - loss: 0.2953 - val_loss: 1.3803
Epoch 220/300
591/591 [==============================] - 781s 1s/step - loss: 0.2070 - val_loss: 0.8149
Epoch 221/300
591/591 [==============================] - 775s 1s/step - loss: 0.2065 - val_loss: 0.9254
Epoch 222/300
591/591 [==============================] - 744s 1s/step - loss: 0.4169 - val_loss: 1.7400
Epoch 223/300
591/591 [==============================] - 743s 1s/step - loss: 0.3119 - val_loss: 1.0373
Epoch 224/300
591/591 [==============================] - 748s 1s/step - loss: 0.6624 - val_loss: 1.5890
Epoch 225/300
591/591 [==============================] - 751s 1s/step - loss: 0.3222 - val_loss: 0.9081
Epoch 226/300
591/591 [==============================] - 752s 1s/step - loss: 0.3027 - val_loss: 5.3075
Epoch 227/300
591/591 [==============================] - 776s 1s

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\1\EfficientNetV2L.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\1\EfficientNetV2L.ckpt\assets


591/591 [==============================] - 1059s 2s/step - loss: 0.3389 - val_loss: 0.3496
Epoch 240/300
591/591 [==============================] - 808s 1s/step - loss: 0.2704 - val_loss: 9.8191
Epoch 241/300
591/591 [==============================] - 789s 1s/step - loss: 0.3270 - val_loss: 0.8710
Epoch 242/300
591/591 [==============================] - 791s 1s/step - loss: 0.1723 - val_loss: 1.8307
Epoch 243/300
591/591 [==============================] - 791s 1s/step - loss: 0.2822 - val_loss: 1.2659
Epoch 244/300
591/591 [==============================] - 792s 1s/step - loss: 0.2602 - val_loss: 1.6760
Epoch 245/300
591/591 [==============================] - 820s 1s/step - loss: 0.3596 - val_loss: 1.1454
Epoch 246/300
591/591 [==============================] - 817s 1s/step - loss: 0.2087 - val_loss: 2.4870
Epoch 247/300
591/591 [==============================] - 818s 1s/step - loss: 0.1808 - val_loss: 0.6694
Epoch 248/300
591/591 [==============================] - 817s 1s/step - loss: